<a href="https://colab.research.google.com/github/phaneendra2429/Mental_Health_Chatbot/blob/Liz/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentence Similarity




In [4]:
!pip install -U sentence-transformers

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import string

#Counting KeyWords
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#install transformers
from sentence_transformers import SentenceTransformer, util
model= SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
nlp = spacy.load("en_core_web_sm")

In [9]:
df_mental_health = pd.read_excel("/content/drive/MyDrive/Team 5/Depression_dataset_preprocessed (1).xlsx", sheet_name= "98_row_Mental_Health_FAQs")
df_counsellor_chats = pd.read_excel("/content/drive/MyDrive/Team 5/Depression_dataset_preprocessed (1).xlsx", sheet_name= "Counsellor_Chats")
df_human_therapist = pd.read_excel("/content/drive/MyDrive/Team 5/Depression_dataset_preprocessed (1).xlsx", sheet_name= "99_rows_Human_&_Therapist")

In [10]:
print(df_mental_health.columns)
print(df_counsellor_chats.columns)
print(df_human_therapist.columns)

Index(['Unnamed: 0', 'Question_ID', 'Questions', 'Answers',
       'Questions_cleaned', 'Answers_cleaned'],
      dtype='object')
Index(['Unnamed: 0', 'questionID', 'questionTitle', 'questionText', 'topic',
       'answerText', 'views', 'therapistInfo'],
      dtype='object')
Index(['Unnamed: 0', 'Human', 'Assistant'], dtype='object')


In [11]:
df_counsellor_chats.head()

,Unnamed: 0,questionID,questionTitle,questionText,topic,answerText,views,therapistInfo
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"If everyone thinks you're worthless, then mayb...",2899,"Sherry Katz, LCSWCouples and Family Therapist,..."
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...",3514,"Robin Landwehr, DBH, LPCC, NCCMental Health in..."
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,First thing I'd suggest is getting the sleep y...,5,Lee KingI use an integrative approach to treat...
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,Therapy is essential for those that are feelin...,31,"Shauntai Davis-YearginPersonalized, private on..."
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,I first want to let you know that you are not ...,620,Jordan WhiteLicensed Social Worker at Oak Root...


In [12]:
df_counsellor_chats.topic.value_counts()

depression                  330
anxiety                     249
counseling-fundamentals     240
intimacy                    205
relationships               174
parenting                   145
family-conflict             116
self-esteem                  83
relationship-dissolution     76
trauma                       66
behavioral-change            51
marriage                     46
lgbtq                        38
anger-management             38
substance-abuse              35
spirituality                 34
professional-ethics          32
workplace-relationships      26
diagnosis                    22
domestic-violence            21
social-relationships         20
grief-and-loss               18
self-harm                    11
sleep-improvement            10
eating-disorders             10
legal-regulatory              8
stress                        7
children-adolescents          6
addiction                     5
human-sexuality               4
military-issues               3
Name: to

In [13]:
#Only have depression related questions
df_counsellor_chats = df_counsellor_chats[df_counsellor_chats.topic == "depression"]

In [14]:
# Group by the questionTitle column and aggregate the answers into a single string
combined_answers = df_counsellor_chats.groupby('questionTitle')['answerText'].agg(lambda x: ' '.join(x)).reset_index()

In [15]:
# Rename the columns if needed
combined_answers.columns = ['questionTitle', 'combined_answers']

# Print the resulting DataFrame
combined_answers.head()

,questionTitle,combined_answers
0,Can I change my feeling of being worthless to ...,"If everyone thinks you're worthless, then mayb..."
1,Can i learn to be happy alone?,Is it possible that even though your illness i...
2,Do I have anxiety?,Everyone has some level of anxiety - it's what...
3,Do I have too many issues for counseling?,Let me start by saying there are never too man...
4,Does my fiancÃ©e need a service dog?,It sounds like you have been a positive suppor...


In [16]:
df1_selected= df_human_therapist[["Human", "Assistant"]]
df2_selected= df_mental_health[["Questions_cleaned", "Answers_cleaned"]]
df3_selected= combined_answers[["questionTitle", "combined_answers"]]

In [17]:
# Rename the columns for consistency
df2_selected.columns = ['Human', 'Assistant']
df3_selected.columns = ['Human', 'Assistant']

In [18]:
# Concatenate the rows of the DataFrames
combined_df = pd.concat([df1_selected, df2_selected, df3_selected], ignore_index=True)


In [19]:
combined_df.shape #327 rows of data

(327, 2)

In [20]:
# Count occurrences of each data type in the "Human" column
assistant_dtype_counts = combined_df['Human'].apply(type).value_counts()

# Print the counts
print("Counts of data types in 'Assistant' column:")
print(assistant_dtype_counts)

Counts of data types in 'Assistant' column:
<class 'str'>      326
<class 'float'>      1
Name: Human, dtype: int64


In [21]:
# Filter rows where the input in the "Human" column is not a float
combined_df = combined_df[~combined_df['Human'].apply(lambda x: isinstance(x, float))]

In [22]:
#Text preprocess with string. Remove punctuations and make lowercase
def column_preprocess(df, columnname):

  def word_preprocess(text):
    text_without_punct = text.translate(str.maketrans(" ", " ", string.punctuation))
    cleaned_text = text_without_punct.lower()
    return cleaned_text
  df[columnname]= df[columnname].apply(word_preprocess)

  return df

In [23]:
cleaned_df = column_preprocess(combined_df, "Human")
cleaned_df = column_preprocess(combined_df, "Assistant")

In [24]:
cleaned_df.reset_index(inplace=True)

In [25]:
sentences = cleaned_df["Human"]
#Encode all sentences
embeddings= model.encode(sentences)
#Compute cosine similarity between all pairs
cos_sim= util.cos_sim(embeddings, embeddings)

In [26]:
similarity_threshold= 0.6

#Add all pairs to a list
grouped_sentences= []

#question-group mapping

questions_to_be_grouped= {}

#counter number for grouping in numbers

group_number =1

#empty list for current group
current_group= []


for i in range(len(cos_sim)):

  #check if the index has already been grouped
  if i not in questions_to_be_grouped:
    current_group = [i]
    for j in range(len(cos_sim)):
      if i!=j and cos_sim[i][j]>=similarity_threshold:
        current_group.append(j)
        questions_to_be_grouped[j]= group_number

    group_number +=1
    if len(current_group) >1:
      grouped_sentences.append(current_group)

In [27]:
#Print the grouped questions with group numbers
for group_number, group in enumerate(grouped_sentences, start=1):
  print(f"Group {group_number}")
  for i in group:
    print(cleaned_df.loc[i, "Human"])
  print('\n')

Group 1
 what are symptoms of panic attack vs anxiety attack 
 what’s the difference between anxiety and stress 
whats the difference between anxiety and stress


Group 2
 what are the types of mental illness 
 how can you treat mental illness 
 what causes mental health problems 
 what are some of the warning signs of mental illness 
 what psychological factors contribute to mental illness 
 how does someone acquire a mental illness 
 where can i learn about types of mental health treatment 
 what types of mental illness and mental health problems can be treated by a psychiatrist 
what does it mean to have a mental illness
what causes mental illness
what are some of the warning signs of mental illness
where can i learn about types of mental health treatment
what are the different types of mental health professionals
what is mental health
what causes mental health problems


Group 3
 how to manage stress 
how can i manage my anxiety


Group 4
 my family has no history of mental illness